<a href="https://colab.research.google.com/github/dip-d10/Twitter_sentiment_analysis/blob/main/S_A_Using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

df = pd.read_csv("/content/twitter_training.csv")

df = df.drop(df.columns[[0, 1]], axis=1)
df.iloc[:5, 0] = "Negative"  # Change 'positive' or 'negative' to 'neutral' (or any value)
df.columns = ['Sentiment','Reviews']
df.head(20)

,Sentiment,Reviews
0,Negative,I am coming to the borders and I will kill you...
1,Negative,im getting on borderlands and i will kill you ...
2,Negative,im coming on borderlands and i will murder you...
3,Negative,im getting on borderlands 2 and i will murder ...
4,Negative,im getting into borderlands and i can murder y...
5,Positive,So I spent a few hours making something for fu...
6,Positive,So I spent a couple of hours doing something f...
7,Positive,So I spent a few hours doing something for fun...
8,Positive,So I spent a few hours making something for fu...
9,Positive,2010 So I spent a few hours making something f...


In [ ]:
import pandas as pd
import re  # Regular expressions for text cleaning
import string  # String functions for punctuation removal
import nltk  # Natural Language Toolkit (for stopwords & lemmatization)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

df['Sentiment'] = df['Sentiment'].str.lower()
df['Reviews'] = df['Reviews'].str.lower()
df.head()

,Sentiment,Reviews
0,negative,i am coming to the borders and i will kill you...
1,negative,im getting on borderlands and i will kill you ...
2,negative,im coming on borderlands and i will murder you...
3,negative,im getting on borderlands 2 and i will murder ...
4,negative,im getting into borderlands and i can murder y...


In [ ]:
import re

def remove_noise(text):
    if not isinstance(text, str):  # Ensure text is a string
        return ""  # Replace NaN or numbers with an empty string

    text = re.sub(r'@\w+', '', text)  # Remove mentions (@username)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df["Reviews"] = df["Reviews"].astype(str).apply(remove_noise)
df["Reviews"] = df["Reviews"].apply(lambda x: re.sub(r"[^\w\s]", '', x))  # Remove punctuation
df["Reviews"] = df["Reviews"].apply(lambda x: re.sub(r"\d+", '', x))  # Remove numbers
df["Reviews"] = df["Reviews"].apply(lambda x: x.strip())  # Remove leading/trailing spaces
df["Reviews"] = df["Reviews"].apply(lambda x: re.sub(r"\s+", " ", x))  # Remove multiple spaces
nltk.download('stopwords')  # Download stopwords
stop_words = set(stopwords.words('english'))  # Load English stopwords

df["Reviews"] = df["Reviews"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

nltk.download('wordnet')  # Download WordNet data
lemmatizer = WordNetLemmatizer()

df["Reviews"] = df["Reviews"].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

df["Reviews"] = df["Reviews"].fillna("")


df.head()

df.to_csv("cleaned_twitter_sentiment_data.csv", index=False)
print("Cleaned data saved successfully!")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Sentiment,Reviews
0,negative,coming border kill
1,negative,im getting borderland kill
2,negative,im coming borderland murder
3,negative,im getting borderland murder
4,negative,im getting borderland murder


In [ ]:

df = pd.read_csv("cleaned_twitter_sentiment_data.csv")
print(df.head())  # Verify the first few rows

df["Reviews"] = df["Reviews"].astype(str)  # Convert everything to string
df["Reviews"] = df["Reviews"].fillna("")   # Replace NaN with empty string


  Sentiment                       Reviews
0  negative            coming border kill
1  negative    im getting borderland kill
2  negative   im coming borderland murder
3  negative  im getting borderland murder
4  negative  im getting borderland murder


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to numerical data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Limit vocabulary to 5000 words
X = tfidf_vectorizer.fit_transform(df["Reviews"])  # Transform text to numerical form
y = df["Sentiment"]  # Target variable

# Split data (80% training, 20% testing)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")


Training set size: (59744, 10000)
Testing set size: (14937, 10000)


In [ ]:
# Let's use Logistic Regression

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)  # Train the model

print("Model training completed! ✅")


Model training completed! ✅


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Evaluate the model

from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Show detailed classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

  irrelevant       0.71      0.54      0.62      2661
    negative       0.74      0.78      0.76      4472
     neutral       0.63      0.69      0.66      3551
    positive       0.72      0.73      0.72      4253

    accuracy                           0.70     14937
   macro avg       0.70      0.69      0.69     14937
weighted avg       0.70      0.70      0.70     14937



In [ ]:
def predict_sentiment(text):
    text = tfidf_vectorizer.transform([text])  # Convert text to TF-IDF format
    prediction = model.predict(text)  # Make prediction
    return prediction[0]

# Example
sample_text = "I love this product! It's amazing."
print(f"Sentiment: {predict_sentiment(sample_text)}")

sample_text = "This is the worst experience ever."
print(f"Sentiment]: {predict_sentiment(sample_text)}")


Sentiment: positive
Sentiment: negative



**Let's try with Other models**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize models
models = {
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear')
}

# Train & evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictions
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    print(f"{name} Accuracy: {accuracy:.2f}")


Naive Bayes Accuracy: 0.68
Random Forest Accuracy: 0.88
SVM Accuracy: 0.74


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [10, 20, None],  # Depth of trees
    'min_samples_split': [2, 5, 10],  # Min samples required to split
    'min_samples_leaf': [1, 2, 4],  # Min samples per leaf
    'max_features': ['sqrt', 'log2']  # Number of features in each split
}

# Initialize the model
rf = RandomForestClassifier(random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

# Train the model
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Train the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the tuned model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Model Accuracy: {accuracy:.2f}")


#Through this code got
#Best Parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
#Optimized Model Accuracy: 0.91

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Train the final model with best parameters
final_rf_model = RandomForestClassifier(
    max_depth=None,
    max_features='log2',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=300,
    random_state=42
)

# Train the model on the full training set
final_rf_model.fit(X_train, y_train)

# Make predictions
y_pred = final_rf_model.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score

print("Final Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Final Model Accuracy: 0.9080806052085425
Classification Report:
               precision    recall  f1-score   support

  irrelevant       0.97      0.85      0.91      2661
    negative       0.94      0.92      0.93      4472
     neutral       0.83      0.94      0.88      3551
    positive       0.92      0.91      0.92      4253

    accuracy                           0.91     14937
   macro avg       0.91      0.90      0.91     14937
weighted avg       0.91      0.91      0.91     14937



In [ ]:
import joblib

# Save the model
joblib.dump(final_rf_model, "sentiment_analysis_model.pkl")

print("Model saved successfully!")


In [14]:
from google.colab import files

# Download the model
files.download("sentiment_analysis_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

# Copy to Google Drive
!cp sentiment_analysis_model.pkl "/content/drive/My Drive/"
print("Model uploaded to Google Drive!")


Mounted at /content/drive
Model uploaded to Google Drive!


In [19]:
# New test sentences
new_texts = [
    "I never expected things to turn out this way.",
    "That was definitely something different.",
    "I guess this is what I was waiting for.",
    "Well, that’s one way to look at it.",
    "I can’t say I’m surprised."
]

# Transform text using the TF-IDF vectorizer
new_texts_transformed = tfidf_vectorizer.transform(new_texts)

# Predict sentiment
predictions = final_rf_model.predict(new_texts_transformed)

# Display results
for text, sentiment in zip(new_texts, predictions):
    print(f"Text: \"{text}\" --> Sentiment: {sentiment}")

Text: "I never expected things to turn out this way." --> Sentiment: negative
Text: "That was definitely something different." --> Sentiment: positive
Text: "I guess this is what I was waiting for." --> Sentiment: irrelevant
Text: "Well, that’s one way to look at it." --> Sentiment: positive
Text: "I can’t say I’m surprised." --> Sentiment: negative


Text: "I never expected things to turn out this way." --> Sentiment: negative
Text: "That was definitely something different." --> Sentiment: positive
Text: "I guess this is what I was waiting for." --> Sentiment: irrelevant
Text: "Well, that’s one way to look at it." --> Sentiment: positive
Text: "I can’t say I’m surprised." --> Sentiment: negative
